In [2]:

from SSD_generate_anchors import generate_ssd_priors
from CustomDataLoader import DataAugmentor
from utility import *
from pathlib import Path
import collections

Using TensorFlow backend.


In [9]:
# root = Path.home()/'data'/'VOCdevkit'/'VOC2007'
root  = Path.home()/'Documents'/'DATASETS'/'VOCdevkit'/'VOC2007'

voc_2007_datafile = root/'ImageSets'/'Main'/'train.txt'
voc_2007_images   = root/'JPEGImages'
voc_2007_annotations = root/'Annotations'

In [10]:
SSDBoxSizes = collections.namedtuple('SSDBoxSizes', ['min', 'max'])

Spec = collections.namedtuple('Spec', ['feature_map_size', 'shrinkage', 'box_sizes', 
                                       'aspect_ratios'])

# the SSD orignal specs
specs = [
    Spec(38, 8, SSDBoxSizes(30, 60), [2]),
    Spec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
    Spec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
    Spec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
    Spec(3, 100, SSDBoxSizes(213, 264), [2]),
    Spec(1, 300, SSDBoxSizes(264, 315), [2])
]

In [18]:
priors = generate_ssd_priors(specs)
testloader = DataAugmentor()

data       = testloader.flow_from_directory(root = root,
                                            data_file=voc_2007_datafile,
                                            target_size=300,
                                            batch_size = 4,
                                            shuffle    = True
                                            )

In [24]:
test = np.array([[0,1,2],[3,4,5],[6,7,8]])
test.shape

(3, 3)

In [25]:
test = test.reshape(3,3,1)
test.shape

(3, 3, 1)

In [26]:
test = test.repeat(10, axis = -1)
test.shape

(3, 3, 10)

In [27]:
test

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]],

       [[3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
        [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]],

       [[6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
        [7, 7, 7, 7, 7, 7, 7, 7, 7, 7],
        [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]]])

In [39]:
def intersect(box_a, box_b):

    """ We resize both tensors to [A,B,2] without new malloc:
    [A,2] -> [A,1,2] -> [A,B,2]
    [B,2] -> [1,B,2] -> [A,B,2]
    Then we compute the area of intersect between box_a and box_b.
    Args:
      box_a: (tensor) bounding boxes, Shape: [A,4].
      box_b: (tensor) bounding boxes, Shape: [B,4].
    Return:
      (tensor) intersection area, Shape: [A,B].
    """
    box_a = np.array(box_a, dtype = np.float32)
    box_b = np.array(box_b, dtype = np.float32) 
    
    A = box_a.shape[0]
    B = box_b.shape[0]
    

    
    min_xy = np.maximum(box_a[:,:2].reshape(A, 1, -1).repeat(B, axis = 1),
                        box_b[:,:2].reshape(1, B, -1).repeat(A, axis = 0))
    
   
    max_xy = np.minimum(box_a[:,2:].reshape(A, 1, -1).repeat(B, axis = 1),
                        box_b[:,2:].reshape(1, B, -1).repeat(A, axis = 0))
    inter = np.clip((max_xy - min_xy), a_min = 0, a_max = None)
    
    return inter[:,:,0] * inter[:,:,1]
   


def jaccard(box_a, box_b):
    
    intersection = intersect(box_a, box_b)
    print('intersection shape', intersection.shape)
    
    A = box_a.shape[0]
    B = box_b.shape[0]
    

    
    # area_box_a = (xmax - xmin) * (ymax - ymin)
    area_box_a  = (box_a[:,2] - box_a[:,0]) * (box_a[:,3] - box_a[:,1])
    
    area_box_a = area_box_a.reshape(-1, 1)
    area_box_a = area_box_a.repeat(B, axis = 1)
    
    # calculate areas of box B
    area_box_b  = (box_b[:,2] - box_b[:,0]) * (box_b[:,3] - box_b[:,1])
    
    area_box_b = area_box_b.reshape(1,-1)
    area_box_b = area_box_b.repeat(A, axis = 0)
    
    print(area_box_a.shape)
    print(area_box_b.shape)
    union       = area_box_a + area_box_b - intersection
    
    iou          = intersection/ union
    return iou

In [43]:
sample = data[0]

image, target = sample
batch_size = image.shape[0]

for i in range(batch_size):
    t = point_form(target[i][:,1:])
    a = intersect(t, priors)
    b = jaccard(t, point_form(priors)) 
    if i == 0:
        break

intersection shape (2, 8732)
(2, 8732)
(2, 8732)


In [44]:
a.shape


(2, 8732)

In [45]:
a

array([[0.        , 0.        , 0.        , ..., 0.03014665, 0.02322824,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.04953602, 0.01305944,
        0.04953602]], dtype=float32)

In [49]:
b

array([[0.        , 0.        , 0.        , ..., 0.05428569, 0.08061013,
        0.06219075],
       [0.        , 0.        , 0.        , ..., 0.26408085, 0.23495507,
        0.28521135]])

In [52]:
np.amax(b, axis = -1, keepdims=True)

array([[0.65976978],
       [0.78622611]])

In [53]:
np.where(b == np.amax(b, axis = -1, keepdims=True))

(array([0, 1]), array([6889, 8638]))

In [55]:
np.unravel_index(b.argmax(axis =-1), b.shape)

(array([0, 0]), array([6889, 8638]))

In [56]:
b[:,6889]

array([0.65976978, 0.        ])

In [57]:
b[:,8638]

array([0.        , 0.78622611])